<a href="https://colab.research.google.com/github/apriandito/bi-15-maret/blob/main/Demo_Material.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
!pip install pyLDAvis

## **Collecting Twitter Data**

In [ ]:
# Import the necessary libraries
import tweepy 
import pandas as pd

In [ ]:
# Authentication
consumer_key = 'fbqT0mBw9ehG9IhmT2XTSb0Ti'
consumer_secret = 'MJaEGWTR396VqFWZQDmnybqGwbvydCTFFJ9KFLtsMH7vMgfmkr'
access_token = '1323866871288877057-mhauIZAemtLfS4gPxWlyPe6EyGuszv'
access_token_secret = 'fDGMlo6xREFKr4gs3qD6GgoetdFeiFz1DdwuKPJHSsaDL'

In [ ]:
# Set up your authentication to Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Create an API object
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
# Collect tweets
tweets = []
for tweet in tweepy.Cursor(api.search_tweets, q='Bima', lang = "id").items(100):
    tweets.append(tweet)

In [ ]:
# Create Pandas dataframe
df_tweet = pd.DataFrame({'time' : [tweet.created_at for tweet in tweets],
                   'screen_name' : [tweet.user.screen_name for tweet in tweets],
                   'text' : [tweet.text for tweet in tweets],
                   'reply_to_screen_name' : [tweet.in_reply_to_screen_name for tweet in tweets],
                   'location' : [tweet.user.location for tweet in tweets]})

# Show tweets data
df_tweet

In [ ]:
# Save tweets data
df_tweet.to_csv('tweets.csv', index=False)

## **Import Data (Tweet Data Collected using Phantombuster**

In [ ]:
# Import Data
df_tweet = pd.read_csv("https://raw.githubusercontent.com/apriandito/bi-15-maret/main/data/result.csv")

In [ ]:
# Show Data
df_tweet

## **Network Analysis**

### **Create Edgelist**

In [ ]:
# Import the necessary libraries
import re

# Create empty edgelist
edge_list = []

# Iterate over each row in the DataFrame
for index, row in df_tweet.iterrows():
    account_name = row['handle']
    tweet = row['text']
    
    # Use regex to find mentions in the tweet
    mentions = re.findall(r'@(\w+)', tweet)
    
    # Create edges between the account_name and each mentioned account
    edges = [(account_name, mention) for mention in mentions]
    
    # Extend the edge list with the newly created edges
    edge_list.extend(edges)

# Create a new DataFrame from the edge list
edgelist = pd.DataFrame(edge_list, columns=['source', 'target'])
edgelist

In [ ]:
# Import the necessary libraries
import networkx as nx

# Create a graph from the edgelist
G = nx.from_pandas_edgelist(edgelist, source='source', target='target')

# Visualize the graph
nx.draw_networkx(G, pos = nx.kamada_kawai_layout(G), font_size = 7, with_labels= True)

### **Network Properties Measurement**

In [ ]:
# Calculate number of nodes
num_nodes = G.number_of_nodes()

# Calculate number of edges
num_edges = G.number_of_edges()

# Calculate density
density = nx.density(G)

# Create a dataframe based on the results
network_properties = pd.DataFrame(data= {'num_nodes':[num_nodes], 
                                 'num_edges':[num_edges],
                                 'density':[density]})
# Show network properties
network_properties

In [ ]:
# Calculate degree centrality
degree_centrality = nx.degree_centrality(G)

# Calculate betweeness centrality
betweenness_centrality = nx.betweenness_centrality(G)

# Calculate closeness centrality
closeness_centrality = nx.closeness_centrality(G)

# Calculate closeness centrality
eigenvector_centrality = nx.closeness_centrality(G)

# Create a dataframe based on the results
centralities = pd.DataFrame([degree_centrality, betweenness_centrality,
                             closeness_centrality, eigenvector_centrality]).T
centralities.columns = ['degree_centrality', 'betweenness_centrality',
                        'closeness_centrality', 'eigenvector_centrality']

# Show centralities
centralities      

### **Modularity**

In [ ]:
# Import the necessary libraries
from community import community_louvain
import matplotlib.pyplot as plt

In [ ]:
# Calculate the modularity
partition = community_louvain.best_partition(G)
values = [partition.get(node) for node in G.nodes()]
partition

In [ ]:
# Visualize the graph
nx.draw_networkx(G, pos = nx.kamada_kawai_layout(G), cmap = plt.get_cmap('jet'), node_color = values, font_size = 7, with_labels= True)

## **Collecting News Data**

In [ ]:
# Import the necessary libraries
import pandas as pd
import requests

In [ ]:
# Specify the News API key and base url
base_url = "http://newsapi.org/v2/"

In [ ]:
# Specify the parameters of the data you want to collect
params = {
    'q': 'artificial intelligence', #keywords to search for in the news
    'sortBy': 'publishedAt', #sort the news by publication date
    'apiKey': 'f2b2e3f10bee4668ba146f9bdff912fd' #specify the News API key
}

In [ ]:
# Call the API and convert the response to json format
response = requests.get(base_url + 'everything', params=params)
data = response.json()

In [ ]:
#create an empty list to store the data
news_data = []

# Loop through each article and extract relevant fields to the list
for article in data['articles']:
    news_data.append({
        'title': article['title'],
        'description': article['description'],
        'content': article['content'],
        'url': article['url'],
        'publishedAt': article['publishedAt'],
        'source': article['source']['name']
    })

# Convert the list of dictionaries to a dataframe
df_news = pd.DataFrame(news_data)

# Show news data
df_news

In [ ]:
# Save news data
df_news.to_csv("news.csv")

## **Sentiment Analysis: VADER (English)**

In [ ]:
# Import the necessary libraries
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer 

In [ ]:
# Download the corpus
nltk.download('vader_lexicon')

In [ ]:
# Create a vader sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Create a list of sentences
sentences = df_news['title']

# Calculate the compound sentiment score of each sentence
scores = [sia.polarity_scores(sentence)['compound'] for sentence in sentences]

In [ ]:
# Import the necessary libraries
import seaborn as sns

# Visualize the compound sentiment using seaborn
sns.distplot(x=scores)

## **Topic Modeling**

In [ ]:
import string
import gensim
import spacy
import nltk

from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora
import pyLDAvis.gensim_models

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
# Select News Title
news_title = df_news['title']

# Tokenize
tokenized_text = [d.lower().split() for d in news_title]

# Remove punctuation
punctuation = string.punctuation
tokenized_text = [[word for word in doc if word not in punctuation] for doc in tokenized_text]

# Lemmatization
lemmatizer = WordNetLemmatizer()
tokenized_text = [[lemmatizer.lemmatize(word) for word in doc] for doc in tokenized_text]

# Remove stopwords
stop_words = stopwords.words('english')
tokenized_text = [[word for word in doc if word not in stop_words] for doc in tokenized_text]

# Create dictionary
dictionary = corpora.Dictionary(tokenized_text)

# Create corpus
corpus = [dictionary.doc2bow(doc) for doc in tokenized_text]

In [ ]:
# Train LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=3,
                                           passes = 100, 
                                           per_word_topics=True)

In [ ]:
# Enable Notebook 
pyLDAvis.enable_notebook()

# Visualize
pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

## **Text Network Analysis**

In [ ]:
def remove_non_text(value):
    return re.sub(r'[^a-zA-Z\s]', '', value)

In [ ]:

# Create dataframe
df_tna = pd.DataFrame(df_news['title'].str.lower().apply(remove_non_text))
df_tna

In [ ]:
# Create empty graph
G = nx.Graph()

In [ ]:
# For each row in dataframe
for index, row in df_tna.iterrows():
    
    # Split tittle into words
    words = row['title'].split(' ')
    
    # Add relationships between words
    for w1 in words:
        for w2 in words:
            if w1 != w2:
                G.add_edge(w1,w2)

In [ ]:
# Visualize the graph
nx.draw(G, font_size = 7, with_labels = True)

## **Sentiment Analysis: IndoBERT**

In [ ]:
# Install Huggingface Transformers
! pip install  huggingface transformers

In [ ]:
# import necessary libraries 
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
# Download Pretrained Model
pretrained= "mdhugol/indonesia-bert-sentiment-classification"

# Set Model and Tokenizer
model = AutoModelForSequenceClassification.from_pretrained(pretrained)
tokenizer = AutoTokenizer.from_pretrained(pretrained)

# Create sentiment classifier using huggingface pipeline
sentiment_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [ ]:
# Get sentiment label for each row in dataframe 
df_tweet['sentiment'] = df_tweet['text'].apply(lambda x: sentiment_analysis(x)[0]['label'])


In [ ]:
# Set Label
label_index = {'LABEL_0': 'positive', 'LABEL_1': 'neutral', 'LABEL_2': 'negative'}

# Replace the values in the sentiment column
df_tweet['sentiment'] = df_tweet['sentiment'].replace(label_index)

# Show Tweet with sentiment
df_tweet

In [ ]:
# Visualise the sentiment distribution
sns.countplot(x ='sentiment', data = df_tweet)